In [1]:
import requests
import os as os
import json
import pandas as pd

os.chdir('c://users/grlurton/Google Drive/elections_niger')

In [35]:
dep_url = 'http://www.ceni-niger.org/ceniProject/jsmap/maps/dept-all.js'
dep_head = 'Highcharts.maps["dept-all"]='

r = requests.get(dep_url)
r = r.text.replace(dep_head, '').replace('\r\n' , '').replace('/*' , '').replace('*/' , '')

dept_map = json.loads(r)

In [3]:
reg_url ='http://www.ceni-niger.org/ceniProject/jsmap/maps/reg_all.js'
reg_head = 'Highcharts.maps["reg-all"] ='

r = requests.get(reg_url)
r = r.text.replace(reg_head, '').replace('\r\n' , '').replace('/*' , '').replace('*/' , '')

reg_map = json.loads(r)

In [54]:
nom_tag = 'DEPART'
id_tag = 'KEY_DEPARTEMENT'

def extract_gps(coordinates):
    x = []
    y = []
    for j in range(len(coordinates)) :
        x.append(coordinates[j][0])
        y.append(coordinates[j][1])
    out = {'x':x , 'y':y}
    return out

def extract_zone(zone_data , nom_tag , id_tag):
    zone_name = zone_data['properties'][nom_tag]
    zone_id = zone_data['properties'][id_tag]
    coordinates = extract_gps(zone_data['geometry']['coordinates'][0])
    out = {'id_zone':zone_id , 'name':zone_name , 'x':coordinates['x'] , 'y':coordinates['y']}
    return out


departements = dept_map['features']
dept_map_out = {}
for i in range(len(departements)) :
    extracted = extract_zone(departements[i] , 'DEPART' , 'KEY_DEPARTEMENT')
    dept_map_out[str(extracted['id_zone'])] = extracted
    
regions = reg_map['features']
reg_map_out = {}
for i in range(len(regions)) :
    extracted = extract_zone(regions[i] , 'Nom_reg' , 'Code_reg')
    reg_map_out[str(extracted['id_zone'])] = extracted

In [55]:
with open('data/dept_map.json', 'w') as outfile:
    json.dump(dept_map_out, outfile)
    
with open('data/reg_map.json', 'w') as outfile:
    json.dump(reg_map_out, outfile)